<a href="https://colab.research.google.com/github/FuturisticLover123/learning/blob/main/ifortex_test/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

Посмотрим данные и обновные характеристики

In [4]:
data = pd.read_csv("/content/student_exam_data.csv")

data.describe()

,Контрольная 1,Контрольная 2,Контрольная 3
count,300.000000,300.000000,300.000000
mean,7.130000,6.936667,6.940000
std,2.003283,2.009839,2.028987
min,4.000000,4.000000,4.000000
25%,5.000000,5.000000,5.000000
50%,7.000000,7.000000,7.000000
75%,9.000000,9.000000,9.000000
max,10.000000,10.000000,10.000000


In [5]:
data.isna().sum()

,0
Контрольная 1,0
Контрольная 2,0
Контрольная 3,0
Сон накануне,0
Настроение,0
Энергетиков накануне,0
Посещаемость занятий,0
Время подготовки,0
Сдал,0


В данных нет пропусков это очень хорошо

In [6]:
data.head()

,Контрольная 1,Контрольная 2,Контрольная 3,Сон накануне,Настроение,Энергетиков накануне,Посещаемость занятий,Время подготовки,Сдал
0,4,4,9,Нет,Хорошее,4+,Низкая,За неделю,Нет
1,9,9,6,Да,Плохое,1,Средняя,Последний час,Да
2,8,6,9,Да,Хорошее,2-3,Средняя,Последний час,Да
3,8,4,9,Да,Хорошее,1,Высокая,Последняя ночь,Да
4,9,10,4,Нет,Нормальное,1,Низкая,Последний час,Да


Заметим что в данных много категориальных признаков. Данных не много поэтому нейронки особого смысла обучать нет + много категориальных признаков поэтому основная модель скорее всего будет градиентным бустингом над деревьями.  

Но для построения бэйслайнов которые не могут работаь с категориальными признаками и некоторой предобработки декодируем их. C помощью one hot encoding

In [7]:
x, y = data.drop(columns = ["Сдал"]), data["Сдал"]

Все столбцы кроме оценок считаем категориальными

In [8]:
cat_columns = x.iloc[:, 3:].columns
df_encoded = pd.get_dummies(x, columns=x.iloc[:, 3:].columns).astype(int)

С помощью Изоляционного леса предобработает анамалии. Порог подбирался вручную

In [9]:
from sklearn.ensemble import IsolationForest
iso = IsolationForest(random_state=0, contamination=0.01)
clean = iso.fit_predict(df_encoded, y)

X_cleaned = x[clean != -1]
X_encoded_clean = df_encoded[clean != -1]

y_cleaned = y[clean != -1]

Из понимания домена делаем вывод что наиболее важные факторы это оценки и посещаемость. Поэтому делаем вывод что это скорее всего действительно аномалии

In [10]:
data.iloc[clean == -1]

,Контрольная 1,Контрольная 2,Контрольная 3,Сон накануне,Настроение,Энергетиков накануне,Посещаемость занятий,Время подготовки,Сдал
7,8,6,10,Нет,Хорошее,4+,Средняя,За несколько дней,Нет
32,9,10,10,Нет,Плохое,4+,Средняя,За неделю,Нет
99,4,10,9,Нет,Плохое,4+,Средняя,За неделю,Нет


Попробуем добавить новых фичей

Из понимания домена можно попробовать сделать вывод что наибольший вклад дают именно оценки. Поэтому попробуем добавить несколько статискик этих данных

In [12]:
X_cleaned.loc[:, "Mean"] = X_encoded_clean[X_encoded_clean.columns[:3]].mean(axis=1)
X_encoded_clean.loc[:, "Mean"] = X_encoded_clean[X_encoded_clean.columns[:3]].mean(axis=1)

Добавили среднее из оценок

In [14]:
X_encoded_clean.loc[:, "Min"] = X_encoded_clean[X_encoded_clean.columns[:3]].min(axis=1)

X_encoded_clean = X_encoded_clean.assign(
    Min=X_encoded_clean[X_encoded_clean.columns[:3]].min(axis=1)
)

Добавили минимальную оценку

Попробуем обучить несколько простых моделей

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [66]:
Xen_train, Xen_test, yen_train, yen_test = train_test_split(X_encoded_clean, y_cleaned)

In [19]:
lr = LogisticRegression()
print(cross_val_score(lr, Xen_train, yen_train, cv=20, scoring='accuracy').mean())

0.8416666666666666


In [22]:
knn = KNeighborsClassifier(n_neighbors=3)
print(cross_val_score(knn, Xen_train, yen_train, cv=20, scoring='accuracy').mean())

0.7973484848484848


In [21]:
nb = GaussianNB()
print(cross_val_score(nb, Xen_train, yen_train, cv=20, scoring='accuracy').mean())

0.8143939393939392


К сожалению простые модели не дали хорошей точности. Их можно былобы потюнить и попытаться поднять точность на пару процентов. Но как мы уже говорили датасет имеет много категориальных фич поэтому имеет смысл хорошо обучить catboost

In [67]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned)
cat_features =  [3,4,5,6,7]

In [24]:
!pip install optuna
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.9 MB/s eta 0:00:00


In [26]:
import optuna
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        "random_strength": trial.suggest_float("random_strength", 1e-3, 10.0),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 10, 50),
    }

    kf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        train_pool = Pool(X_tr, y_tr, cat_features=cat_features)
        val_pool = Pool(X_val, y_val, cat_features=cat_features)

        model = CatBoostClassifier(**params, verbose=0)
        model.fit(train_pool, eval_set=val_pool)

        y_pred = model.predict_proba(X_val)[:, 1]
        scores.append(log_loss(y_val, y_pred))

    return np.mean(scores)

Для подбора гиперпараметров используем optuna с 20 разбиением на 20 фолдов

In [27]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, timeout=3600)

[I 2025-04-26 08:31:28,620] A new study created in memory with name: no-name-73a1ecb7-00c0-4e72-bf67-87cdd279cc14
[I 2025-04-26 08:31:38,799] Trial 0 finished with value: 0.21907939839418974 and parameters: {'iterations': 289, 'learning_rate': 0.014202818914613816, 'depth': 10, 'l2_leaf_reg': 0.9801376295496532, 'random_strength': 2.129429886452206, 'bagging_temperature': 0.7945429472262204, 'min_data_in_leaf': 15, 'grow_policy': 'Lossguide', 'early_stopping_rounds': 43}. Best is trial 0 with value: 0.21907939839418974.
[I 2025-04-26 08:31:39,897] Trial 1 finished with value: 0.3560458340300935 and parameters: {'iterations': 163, 'learning_rate': 0.03373422376758068, 'depth': 6, 'l2_leaf_reg': 0.12860440291099767, 'random_strength': 8.799786454080117, 'bagging_temperature': 0.2218821270002408, 'min_data_in_leaf': 13, 'grow_policy': 'Depthwise', 'early_stopping_rounds': 19}. Best is trial 0 with value: 0.21907939839418974.
[I 2025-04-26 08:31:44,458] Trial 2 finished with value: 0.21977

In [68]:
best_params = study.best_params
print("Best params:", best_params)

X_train, X_val, y_train, y_val = train_test_split(X_train,y_train)

Best params: {'iterations': 854, 'learning_rate': 0.2024065838954682, 'depth': 4, 'l2_leaf_reg': 0.6654280526726094, 'random_strength': 3.041999905249156, 'bagging_temperature': 0.13112321511725977, 'min_data_in_leaf': 76, 'grow_policy': 'SymmetricTree', 'early_stopping_rounds': 24}


Смотрим лучшие параметры и добавляем валиданионнцю выборку

In [95]:
final_model = CatBoostClassifier(**best_params,  verbose=100)
final_model.fit(
    Pool(X_train, y_train, cat_features=cat_features),
    eval_set=Pool(X_val, y_val, cat_features=cat_features),
    early_stopping_rounds=50
)

0:	learn: 0.6594089	test: 0.6578169	best: 0.6578169 (0)	total: 2.65ms	remaining: 2.26s
100:	learn: 0.0097788	test: 0.0674885	best: 0.0674885 (100)	total: 464ms	remaining: 3.46s
200:	learn: 0.0027658	test: 0.0522241	best: 0.0517951 (193)	total: 812ms	remaining: 2.64s
300:	learn: 0.0018877	test: 0.0481172	best: 0.0481030 (285)	total: 1.3s	remaining: 2.39s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.04757073189
bestIteration = 349

Shrink model to first 350 iterations.


Благодаря early stopping удалось урезать количество деревьев и помогает избегать переобучения

In [97]:
from sklearn.metrics import accuracy_score
print(accuracy_score(final_model.predict(X_cleaned), y_cleaned))
print(accuracy_score(final_model.predict(X_test), y_test))

0.9831649831649831
0.96


Основная метрика на train и test высокие. Нет переобчучения

In [101]:
#final_model.save_model("model.cmd")

In [ ]:
loaded = CatBoostClassifier()
loaded.load_model("/content/model.cmd")
print(accuracy_score(loaded.predict(X_cleaned), y_cleaned))
print(accuracy_score(loaded.predict(X_test), y_test))